In [ ]:
#1
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.losses import Huber

# Load datasets (replace file paths with actual locations of your datasets)
data_2019_2023 = pd.read_excel("top 5+ stock 2019-2023.xlsx")
data_2024_2033 = pd.read_excel("top 5 2024_2033.xlsx")

# Extract relevant data
repurchase_data = data_2019_2023[data_2019_2023['Parameters'] == 'repurchase of common stock']
top_5_parameters = data_2019_2023[data_2019_2023['Parameters'] != 'repurchase of common stock']

# Reshape and align datasets
# Melt datasets to align by year
top_5_data_flattened = top_5_parameters.melt(id_vars=['Company_name', 'Parameters'], 
                                             var_name='Year', 
                                             value_name='Value')
repurchase_flattened = repurchase_data.melt(id_vars=['Company_name', 'Parameters'], 
                                            var_name='Year', 
                                            value_name='Repurchase')

# Merge datasets
merged_data = pd.merge(top_5_data_flattened, repurchase_flattened, on=['Company_name', 'Year'])
merged_data.rename(columns={'Parameters_x': 'Parameters', 'Parameters_y': 'Repurchase_Parameter'}, inplace=True)

# Compute top 5 parameters by correlation
correlations = (
    merged_data.groupby('Parameters')
    .apply(lambda group: group['Value'].corr(group['Repurchase']))
    .sort_values(ascending=False)
    .head(5)
)
top_5_selected_params = correlations.index.tolist()

# Filter data for top 5 parameters
filtered_data = merged_data[merged_data['Parameters'].isin(top_5_selected_params)]

# Prepare features and target
data_pivoted = filtered_data.pivot_table(index=['Company_name', 'Year'], 
                                         columns='Parameters', 
                                         values='Value')
target = filtered_data.drop_duplicates(subset=['Company_name', 'Year']).set_index(['Company_name', 'Year'])['Repurchase']

# Align features and target
data_pivoted, target = data_pivoted.align(target, join='inner', axis=0)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(data_pivoted, target, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build an enhanced neural network model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    BatchNormalization(),  # Normalize activations
    Dropout(0.3),  # Add dropout for regularization
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(1)  # Output layer for regression
])

# Compile the model with Huber loss
model.compile(optimizer='adam', loss=Huber(), metrics=['mae'])

# Train the model
model.fit(X_train_scaled, y_train, epochs=300, batch_size=16, validation_split=0.2, verbose=1)

# Evaluate the model
test_loss, test_mae = model.evaluate(X_test_scaled, y_test, verbose=1)
print(f"Test Loss: {test_loss}, Test MAE: {test_mae}")

# Load forecast data for 2024-2033
forecast_data_flattened = data_2024_2033.melt(id_vars=['Company_name', 'Parameters'], 
                                              var_name='Year', 
                                              value_name='Value')
forecast_filtered = forecast_data_flattened[forecast_data_flattened['Parameters'].isin(top_5_selected_params)]

# Prepare features for prediction
forecast_features = forecast_filtered.pivot_table(index=['Company_name', 'Year'], 
                                                   columns='Parameters', 
                                                   values='Value')

# Standardize forecast features
forecast_features_scaled = scaler.transform(forecast_features)

# Make predictions
predictions = model.predict(forecast_features_scaled)

# Prepare output
forecast_features['Predicted_Repurchase'] = predictions
forecast_features.reset_index(inplace=True)
forecast_features = forecast_features[['Company_name', 'Year', 'Predicted_Repurchase']]

# Save predictions to CSV
forecast_features.to_csv("predicted_repurchase_2024_2033.csv", index=False)
print("Predictions saved to 'predicted_repurchase_2024_2033.csv'")


In [ ]:
#2
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.losses import Huber

# Load datasets (Replace paths with actual file locations)
data_2019_2023 = pd.read_excel("top 5+ stock 2019-2023.xlsx")
data_2024_2033 = pd.read_excel("top 5 2024_2033.xlsx")

# Extract repurchase data (Target Variable)
repurchase_data = data_2019_2023[data_2019_2023['Parameters'] == 'repurchase of common stock']

# Extract financial parameters (Features)
top_5_parameters = data_2019_2023[data_2019_2023['Parameters'] != 'repurchase of common stock']

# Reshape datasets for merging
repurchase_flattened = repurchase_data.melt(id_vars=['Company_name', 'Parameters'], 
                                            var_name='Year', 
                                            value_name='Repurchase')

top_5_data_flattened = top_5_parameters.melt(id_vars=['Company_name', 'Parameters'], 
                                             var_name='Year', 
                                             value_name='Value')

# Merge features with repurchase data
merged_data = pd.merge(top_5_data_flattened, repurchase_flattened, on=['Company_name', 'Year'])

# Rename columns for clarity
merged_data.rename(columns={'Parameters_x': 'Parameters', 'Parameters_y': 'Repurchase_Parameter'}, inplace=True)

# Compute top 5 parameters most correlated with repurchase
correlations = (
    merged_data.groupby('Parameters')
    .apply(lambda group: group['Value'].corr(group['Repurchase']))
    .sort_values(ascending=False)
    .head(5)  # Select top 5 parameters
)

# Get top 5 selected parameters
top_5_selected_params = correlations.index.tolist()

# Filter dataset using selected parameters
filtered_data = merged_data[merged_data['Parameters'].isin(top_5_selected_params)]

# Pivot data to match ML input format
data_pivoted = filtered_data.pivot_table(index=['Company_name', 'Year'], 
                                         columns='Parameters', 
                                         values='Value')

# Extract target (Repurchase values)
target = filtered_data.drop_duplicates(subset=['Company_name', 'Year']).set_index(['Company_name', 'Year'])['Repurchase']

# Align features and target properly
data_pivoted, target = data_pivoted.align(target, join='inner', axis=0)

# Handle missing values (Forward Fill + Mean Imputation)
data_pivoted = data_pivoted.fillna(method='ffill').fillna(data_pivoted.mean())

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(data_pivoted, target, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build the neural network model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(1)  # Output layer for regression
])

# Compile model with Huber loss (handles outliers better)
model.compile(optimizer='adam', loss=Huber(), metrics=['mae'])

# Train the model
model.fit(X_train_scaled, y_train, epochs=300, batch_size=16, validation_split=0.2, verbose=1)

# Evaluate model on test data
test_loss, test_mae = model.evaluate(X_test_scaled, y_test, verbose=1)
print(f"Test Loss: {test_loss}, Test MAE: {test_mae}")

# --------------------------------
# STEP 2: Forecasting for 2024-2033
# --------------------------------

# Prepare forecast dataset
forecast_flattened = data_2024_2033.melt(id_vars=['Company_name', 'Parameters'], 
                                         var_name='Year', 
                                         value_name='Value')

# Filter forecast data using top 5 selected parameters
forecast_filtered = forecast_flattened[forecast_flattened['Parameters'].isin(top_5_selected_params)]

# Pivot forecast data to match model input format
forecast_features = forecast_filtered.pivot_table(index=['Company_name', 'Year'], 
                                                  columns='Parameters', 
                                                  values='Value')

# Handle missing values in forecast dataset
forecast_features = forecast_features.fillna(method='ffill').fillna(forecast_features.mean())

# Standardize forecast data using trained scaler
forecast_features_scaled = scaler.transform(forecast_features)

# Make predictions for 2024-2033
predictions = model.predict(forecast_features_scaled)

# Prepare output dataframe
forecast_features['Predicted_Repurchase'] = predictions
forecast_features.reset_index(inplace=True)
forecast_results = forecast_features[['Company_name', 'Year', 'Predicted_Repurchase']]

# Save corrected predictions to CSV
forecast_results.to_csv("corrected_predicted_repurchase_2024_2033.csv", index=False)
print("✅ Predictions saved to 'corrected_predicted_repurchase_2024_2033.csv'")


C:\Users\HP\AppData\Local\Temp\ipykernel_3516\3512372816.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda group: group['Value'].corr(group['Repurchase']))
C:\Users\HP\AppData\Local\Temp\ipykernel_3516\3512372816.py:59: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data_pivoted = data_pivoted.fillna(method='ffill').fillna(data_pivoted.mean())
C:\Users\HP\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first l

Epoch 1/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - loss: 1187.3820 - mae: 1187.8820 - val_loss: 29.5960 - val_mae: 29.9787
Epoch 2/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step - loss: 1187.1602 - mae: 1187.6602 - val_loss: 29.5927 - val_mae: 29.9792
Epoch 3/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 448ms/step - loss: 1187.3175 - mae: 1187.8175 - val_loss: 29.5910 - val_mae: 29.9811
Epoch 4/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 239ms/step - loss: 1187.6307 - mae: 1188.1307 - val_loss: 29.5909 - val_mae: 29.9831
Epoch 5/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/step - loss: 1187.1835 - mae: 1187.6835 - val_loss: 29.5912 - val_mae: 29.9855
Epoch 6/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step - loss: 1186.9894 - mae: 1187.4894 - val_loss: 29.5907 - val_mae: 29.9876
Epoch 7/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/step - loss: 1186.5344 - mae: 1187.0344 - val_loss: 29.5922 - val_mae: 29.9906
Epoch 8/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step - loss: 1186.5829 - mae: 1187.0829 - val_loss: 29.5941 - val_mae: 29.9931
Epo

C:\Users\HP\AppData\Local\Temp\ipykernel_3516\3512372816.py:109: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  forecast_features = forecast_features.fillna(method='ffill').fillna(forecast_features.mean())


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step
✅ Predictions saved to 'corrected_predicted_repurchase_2024_2033.csv'


In [ ]:
#3
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.losses import Huber

# Load datasets (Replace paths with actual file locations)
data_2019_2023 = pd.read_excel("top 5+ stock 2019-2023.xlsx")
data_2024_2033 = pd.read_excel("top 5 2024_2033.xlsx")

# Extract repurchase data (Target Variable)
repurchase_data = data_2019_2023[data_2019_2023['Parameters'] == 'repurchase of common stock']

# Extract financial parameters (Features)
top_5_parameters = data_2019_2023[data_2019_2023['Parameters'] != 'repurchase of common stock']

# Reshape datasets for merging
repurchase_flattened = repurchase_data.melt(id_vars=['Company_name', 'Parameters'], 
                                            var_name='Year', 
                                            value_name='Repurchase')

top_5_data_flattened = top_5_parameters.melt(id_vars=['Company_name', 'Parameters'], 
                                             var_name='Year', 
                                             value_name='Value')

# Merge features with repurchase data
merged_data = pd.merge(top_5_data_flattened, repurchase_flattened, on=['Company_name', 'Year'])

# Rename columns for clarity
merged_data.rename(columns={'Parameters_x': 'Parameters', 'Parameters_y': 'Repurchase_Parameter'}, inplace=True)

# Compute top 5 parameters most correlated with repurchase
correlations = (
    merged_data.groupby('Parameters')
    .apply(lambda group: group['Value'].corr(group['Repurchase']))
    .sort_values(ascending=False)
    .head(5)  # Select top 5 parameters
)

# Get top 5 selected parameters
top_5_selected_params = correlations.index.tolist()

# Filter dataset using selected parameters
filtered_data = merged_data[merged_data['Parameters'].isin(top_5_selected_params)]

# Pivot data to match ML input format
data_pivoted = filtered_data.pivot_table(index=['Company_name', 'Year'], 
                                         columns='Parameters', 
                                         values='Value')

# Extract target (Repurchase values)
target = filtered_data.drop_duplicates(subset=['Company_name', 'Year']).set_index(['Company_name', 'Year'])['Repurchase']

# Align features and target properly
data_pivoted, target = data_pivoted.align(target, join='inner', axis=0)

# Handle missing values (Forward Fill + Mean Imputation)
data_pivoted = data_pivoted.fillna(method='ffill').fillna(data_pivoted.mean())

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(data_pivoted, target, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build the neural network model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(1)  # Output layer for regression
])

# Compile model with Huber loss (handles outliers better)
model.compile(optimizer='adam', loss=Huber(), metrics=['mae'])

# Train the model
model.fit(X_train_scaled, y_train, epochs=300, batch_size=16, validation_split=0.2, verbose=1)

# Evaluate model on test data
test_loss, test_mae = model.evaluate(X_test_scaled, y_test, verbose=1)
print(f"Test Loss: {test_loss}, Test MAE: {test_mae}")

# --------------------------------
# STEP 2: Forecasting for 2024-2033
# --------------------------------

# Prepare forecast dataset
forecast_flattened = data_2024_2033.melt(id_vars=['Company_name', 'Parameters'], 
                                         var_name='Year', 
                                         value_name='Value')

# Filter forecast data using top 5 selected parameters
forecast_filtered = forecast_flattened[forecast_flattened['Parameters'].isin(top_5_selected_params)]

# Pivot forecast data to match model input format
forecast_features = forecast_filtered.pivot_table(index=['Company_name', 'Year'], 
                                                  columns='Parameters', 
                                                  values='Value')

# Handle missing values in forecast dataset
forecast_features = forecast_features.fillna(method='ffill').fillna(forecast_features.mean())

# Standardize forecast data using trained scaler
forecast_features_scaled = scaler.transform(forecast_features)

# Make predictions for 2024-2033
predictions = model.predict(forecast_features_scaled)

# Prepare output dataframe
# Convert predictions to integers (rounding)
forecast_features['Predicted_Repurchase'] = np.round(predictions).astype(int)

# Save the corrected integer predictions
forecast_features.reset_index(inplace=True)
forecast_results = forecast_features[['Company_name', 'Year', 'Predicted_Repurchase']]
forecast_results.to_csv("final_corrected_predicted_repurchase_2024_2033.csv", index=False)

print("✅ Predictions saved as integers in 'final_corrected_predicted_repurchase_2024_2033.csv'")

C:\Users\HP\AppData\Local\Temp\ipykernel_3516\270809271.py:39: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda group: group['Value'].corr(group['Repurchase']))
C:\Users\HP\AppData\Local\Temp\ipykernel_3516\270809271.py:62: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data_pivoted = data_pivoted.fillna(method='ffill').fillna(data_pivoted.mean())
C:\Users\HP\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first lay

Epoch 1/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - loss: 1187.2588 - mae: 1187.7588 - val_loss: 29.6689 - val_mae: 30.0520
Epoch 2/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 227ms/step - loss: 1186.8270 - mae: 1187.3270 - val_loss: 29.6636 - val_mae: 30.0450
Epoch 3/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 213ms/step - loss: 1186.3450 - mae: 1186.8450 - val_loss: 29.6561 - val_mae: 30.0348
Epoch 4/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 502ms/step - loss: 1186.6486 - mae: 1187.1486 - val_loss: 29.6504 - val_mae: 30.0269
Epoch 5/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 202ms/step - loss: 1186.3241 - mae: 1186.8241 - val_loss: 29.6450 - val_mae: 30.0207
Epoch 6/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 228ms/step - loss: 1186.2577 - mae: 1186.7577 - val_loss: 29.6376 - val_mae: 30.0153
Epoch 7/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step - loss: 1186.0521 - mae: 1186.5521 - val_loss: 29.6271 - val_mae: 30.0086
Epoch 8/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 201ms/step - loss: 1186.2753 - mae: 1186.7753 - val_loss: 29.6181 - val_mae: 30.0041
Epo

C:\Users\HP\AppData\Local\Temp\ipykernel_3516\270809271.py:112: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  forecast_features = forecast_features.fillna(method='ffill').fillna(forecast_features.mean())


1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/stepWARNING:tensorflow:6 out of the last 6 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x000001DACF06E0C0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 196ms/step
✅ Predictions saved as integers in 'final_corrected_predicted_repurchase_2024_2033.csv'
